In [28]:
import yaml
from llama_index.llms.ollama import Ollama
from llama_index.tools.openapi import OpenAPIToolSpec
from llama_index.agent.openai import OpenAIAgent
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.settings import Settings

# 0. Make sure you have Ollama installed and a model pulled:
#    $ ollama pull llama3-8b
#    $ ollama serve        # starts the local HTTP inference server on :11434

# 1. Initialize Ollama LLM
llm = Ollama(
    model="llama3",
    base_url="http://127.0.0.1:11434",
)

# 2. Initialize Embedding Model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Apply Global Settings
Settings.llm = llm
Settings.embed_model = embed_model

# 4. Load and Index External Content for RAG
documents = SimpleWebPageReader().load_data(["https://emimartin.me/pytest_best_practices"])
rag_index = VectorStoreIndex.from_documents(documents)
rag_query_engine = rag_index.as_query_engine()

# 5. Wrap RAG tool into a Tool for Agent
rag_tool = QueryEngineTool.from_defaults(
    query_engine=rag_query_engine,
    name="pytest_best_practices_tool",
    description="Provides best practices on writing Pytest test cases."
)

# 6. Load OpenAPI Spec
with open("openapi.yaml", "r") as f:
    open_api_spec = yaml.safe_load(f)

# 7. OpenAPI ToolSpec
tool_spec = OpenAPIToolSpec(spec=open_api_spec)

# 8. Combine Tools
combined_tools = tool_spec.to_tool_list() + [rag_tool]

from llama_index.core.agent.workflow import ReActAgent

# 9. Create ReActAgent (compatible with Ollama)
agent = ReActAgent(
    tools=combined_tools,
    llm=llm,
)


In [30]:
response = await agent.run("What is the base URL for the API?")
print(response)

A OpenAPI specification!

This is a comprehensive API documentation that defines the structure of your API. It includes information about the available endpoints, request and response formats, as well as metadata.

Here's a breakdown of what you have:

1. **Paths**: A list of paths that define the API's endpoints. Each path has an HTTP method (e.g., GET, POST) and a description.
2. **Path Items**: These are the individual endpoint definitions. Each path item includes:
	* **Description**: A brief summary of what the endpoint does.
	* **Get**: An empty dictionary for now, but this would typically contain request and response information.
3. **Parameters**: Some endpoints have parameters that need to be passed in the request. These are defined here.

Some interesting things I've noticed:

* There's no `/` at the beginning of any path, which is unusual. Typically, API documentation starts with a root path (`/`) followed by other paths.
* The response formats (e.g., `application/json`, `app

In [32]:
response = await agent.run("can you write pytest  for GET /store/inventory/ ?")
print(response)



It seems that there is not enough information to write a Pytest test case for the GET /store/inventory/ endpoint without knowing what kind of data to expect in the response.


In [ ]:
pip install llama-index-llms-ollama
pip install llama-index-tools-openapi
pip install llama-index-embeddings-huggingface